In [1]:
import os
import requests
import pyttsx3


In [2]:
JOKE_API_URL = "https://v2.jokeapi.dev/joke/"

def get_joke(category: str) -> str:
    """
    Fetches a joke from the API based on the selected category.

    Args:
        category (str): The category of the joke (e.g., 'Programming', 'Pun').

    Returns:
        str: The joke in the format "Setup ... Delivery".
    """
    url = f"{JOKE_API_URL}{category}?type=twopart"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return f"{data['setup']} ... {data['delivery']}"
    else:
        return "Oops! Couldn't fetch a joke."

In [3]:
get_joke("programming")

'Why do programmers prefer using the dark mode? ... Because light attracts bugs.'

In [4]:


def save_joke(joke: str, filename: str) -> None:
    """
    Saves the joke to a text file.

    Args:
        joke (str): The joke text.
        filename (str): The name of the file to save the joke in. Default is 'joke.txt'.
    """
    with open(filename, "w") as file:
        file.write(joke)
    print(f"Joke saved to {filename}")



In [5]:
save_joke("This is a joke", "joke.txt")

Joke saved to joke.txt


In [6]:


def set_volume(level: int) -> None:
    """
    Sets the system volume using pactl.
    
    Args:
        level (int): Volume level in percentage (0-100).
    """
    os.system(f"pactl set-sink-volume @DEFAULT_SINK@ {level}%")
    print(f"Volume set to {level}%")



In [14]:
def set_brightness(percentage):
    """
    Adjust screen brightness on Ubuntu by percentage (0-100)
    
    Args:
        percentage (int): Desired brightness level (0-100)
    
    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Validate input
        if not 0 <= percentage <= 100:
            raise ValueError("Brightness must be between 0 and 100")

        # Common path for backlight control (might vary by hardware)
        backlight_path = "/sys/class/backlight/intel_backlight/"
        
        # Some systems might use different backlight drivers
        import os
        if not os.path.exists(backlight_path):
            # Try AMD GPU path
            backlight_path = "/sys/class/backlight/amdgpu_bl0/"
        
        if not os.path.exists(backlight_path):
            print("Could not find backlight control path")
            return False

        # Get maximum brightness value
        with open(f"{backlight_path}max_brightness", "r") as f:
            max_brightness = int(f.read().strip())

        # Calculate target brightness
        target_brightness = int((percentage / 100) * max_brightness)

        # Set brightness
        with open(f"{backlight_path}brightness", "w") as f:
            f.write(str(target_brightness))

        return True

    except PermissionError:
        print("Permission denied: Try running with sudo")
        return False
    except FileNotFoundError:
        print("Backlight control files not found - hardware may not support this method")
        return False
    except Exception as e:
        print(f"Error setting brightness: {str(e)}")
        return False

# # Example usage
# if __name__ == "__main__":
#     # Set brightness to 75%
#     success = set_brightness(10)
#     if success:
#         print("Brightness adjusted successfully")

In [7]:
set_volume(45)

Volume set to 45%


In [8]:
def tell_joke(filename: str) -> None:
    """
    Reads the joke aloud using text-to-speech (TTS).

    Args:
        filename (str): The file from which the joke will be read. Default is 'joke.txt'.
    """
    with open(filename, "r") as file:
        joke = file.read()

    tts = pyttsx3.init()
    tts.say("Here is a joke for you!   ")
    tts.say(joke)
    tts.runAndWait()

In [9]:
tell_joke("joke.txt")

In [ ]:
# Install the google-genai package
%pip install google-genai

# Import the required class
from google.genai.types import GenerateContentConfig

Exception ignored on calling ctypes callback function: <bound method EspeakDriver._onSynth of <pyttsx3.drivers.espeak.EspeakDriver object at 0x74e8bcefd9a0>>
Traceback (most recent call last):
  File "/home/ofge/Documents/icog/FunctionCall/fun/lib/python3.12/site-packages/pyttsx3/drivers/espeak.py", line 193, in _onSynth
    self._proxy.notify('finished-utterance', completed=True)
    ^^^^^^^^^^^^^^^^^^
ReferenceError: weakly-referenced object no longer exists


Note: you may need to restart the kernel to use updated packages.


In [11]:
from google import genai
 
# create client
api_key = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)
 
# Define the model you are going to use
model_id =  "gemini-2.0-flash"


In [12]:


 
# Generation Config
config = GenerateContentConfig(
    system_instruction="You are a helpful assistant that use tools to provide jokes to the user", # to give the LLM context.
    tools=[get_joke, save_joke, set_volume, tell_joke, ], # define the functions that the LLM can use
)



In [16]:


r = client.models.generate_content(
    model=model_id,
    config=config,
    contents='tell me a joke about programming, and save it in a file named "joke.txt", then set_brightness 75%'
)

# set the volume "50" percent
# tell me a joke about programming
# tell me a joke about programming, and save it in a file named "joke.txt", then read it aloud


print(r.text)

I am sorry, I cannot set the brightness. I can tell you a joke about programming and save it in a file named "joke.txt". Do you want me to do that?


In [ ]:


r = client.models.generate_content(
    model=model_id,
    config=config,
    contents='tell me a joke about programming, and save it in a file named "joke.txt", then read it aloud'
)

# set the volume "50" percent
# tell me a joke about programming
# tell me a joke about programming, and save it in a file named "joke.txt", then read it aloud

set_brightness
print(r.text)

Joke saved to joke.txt
OK. I saved the joke "Why did the web developer walk out of a resturant in disgust? ... The seating was laid out in tables." to joke.txt and read it aloud.



Exception ignored on calling ctypes callback function: <bound method EspeakDriver._onSynth of <pyttsx3.drivers.espeak.EspeakDriver object at 0x74e8a62215e0>>
Traceback (most recent call last):
  File "/home/ofge/Documents/icog/FunctionCall/fun/lib/python3.12/site-packages/pyttsx3/drivers/espeak.py", line 193, in _onSynth
    self._proxy.notify('finished-utterance', completed=True)
    ^^^^^^^^^^^^^^^^^^
ReferenceError: weakly-referenced object no longer exists


In [1]:
import os
import requests
import subprocess
import pyttsx3
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from textblob import TextBlob
from google.genai.types import GenerateContentConfig
from google import genai

# Load environment variables
load_dotenv()

# API Keys & Config
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = "http://localhost:8080"

# --- Initialize Spotify Client ---
def initialize_spotify():
    """Authenticates and returns a Spotify client instance."""
    try:
        return spotipy.Spotify(auth_manager=SpotifyOAuth(
            client_id=SPOTIFY_CLIENT_ID,
            client_secret=SPOTIFY_CLIENT_SECRET,
            redirect_uri=SPOTIFY_REDIRECT_URI,
            scope="playlist-read-private"
        ))
    except Exception as e:
        print(f"Spotify authentication failed: {e}")
        return None

sp = initialize_spotify()

# --- Brightness Control ---
def set_brightness(level: int) -> None:
    """
    Sets the system brightness level.

    Args:
        level (int): Brightness percentage (0-100).
    """
    try:
        if not 0 <= level <= 100:
            raise ValueError("Brightness must be between 0 and 100.")

        backlight_path = "/sys/class/backlight/intel_backlight/"
        if not os.path.exists(backlight_path):
            backlight_path = "/sys/class/backlight/amdgpu_bl0/"

        if not os.path.exists(backlight_path):
            print("Could not find backlight control path.")
            return

        with open(f"{backlight_path}max_brightness", "r") as f:
            max_brightness = int(f.read().strip())

        target_brightness = int((level / 100) * max_brightness)

        cmd = f"echo {target_brightness} | sudo tee {backlight_path}brightness"
        subprocess.run(cmd, shell=True, capture_output=True, text=True)

        print(f"Brightness set to {level}%")
    except Exception as e:
        print(f"Error setting brightness: {str(e)}")

# --- Volume Control ---
def set_volume(level: int) -> None:
    """
    Sets the system volume.

    Args:
        level (int): Volume percentage (0-100).
    """
    os.system(f"pactl set-sink-volume @DEFAULT_SINK@ {level}%")
    print(f"Volume set to {level}%")

# --- Mood Detection ---
def detect_mood(text: str) -> str:
    """
    Analyzes text and determines mood.

    Args:
        text (str): Input text.

    Returns:
        str: Detected mood ('happy', 'sad', 'stressed', 'neutral', 'angry').
    """
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    text_lower = text.lower()

    if "angry" in text_lower or "mad" in text_lower:
        return "angry"
    elif "happy" in text_lower or "joy" in text_lower:
        return "happy"
    elif "sad" in text_lower or "depressed" in text_lower:
        return "sad"
    elif "stressed" in text_lower or "anxious" in text_lower:
        return "stressed"

    return "happy" if polarity > 0.3 else "sad" if polarity < -0.3 else "neutral"

# --- Spotify Music Recommendation ---
MOOD_PLAYLISTS = {
    "happy": "spotify:playlist:37i9dQZF1DXdPec7aLTmlC",
    "sad": "spotify:playlist:37i9dQZF1DX64Y3ftTBfaN",
    "stressed": "spotify:playlist:37i9dQZF1DWU0ScTcjJBdj",
    "neutral": "spotify:playlist:37i9dQZF1DX0XUsuxWHRQd",
    "angry": "spotify:playlist:37i9dQZF1DX2DCrI7t4mmy"
}

def recommend_music(mood: str) -> str:
    """
    Recommends a Spotify playlist based on mood.

    Args:
        mood (str): User's detected mood.

    Returns:
        str: Playlist link.
    """
    if not sp:
        return "Spotify integration not available."

    playlist_uri = MOOD_PLAYLISTS.get(mood, MOOD_PLAYLISTS["neutral"])
    try:
        playlist = sp.playlist(playlist_uri, market="US")
        return f"Based on your mood, listen to: {playlist['name']} - {playlist['external_urls']['spotify']}"
    except Exception as e:
        return f"Error fetching playlist: {str(e)}"

# --- Text-to-Speech (Joke Telling) ---
def tell_joke(filename: str) -> None:
    """
    Reads a joke aloud using text-to-speech.

    Args:
        filename (str): File containing the joke.
    """
    try:
        with open(filename, "r") as file:
            joke = file.read()

        tts = pyttsx3.init()
        tts.say("Here is a joke for you!")
        tts.say(joke)
        tts.runAndWait()
    except Exception as e:
        print(f"Error reading joke: {str(e)}")

# --- Integrating Gemini AI ---
# Create Gemini Client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define the Model
model_id = "gemini-2.0-flash"

# Generation Config
config = GenerateContentConfig(
    system_instruction="You are a helpful assistant that controls device settings and recommends music based on mood.",
    tools=[set_brightness, set_volume, detect_mood, recommend_music, tell_joke],  # Define callable functions
)

# --- Example Usage ---
if __name__ == "__main__":
    r = client.models.generate_content(
        model=model_id,
        config=config,
        contents='Set the volume to 50%, then recommend music for the mood: "I feel happy."'
    )

    print(r.text)


ValueError: Missing key inputs argument! To use the Google AI API,provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.